In [1]:
import os, sys
import time
import argparse
import multiprocessing
from tqdm import tqdm
import numpy as np
import pandas as pd
from utils.io import compactM, pooling,readcoo2mat,spreadM
from all_parser import *


import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from scipy.sparse import coo_matrix

import models.deephic as deephic

import cooler

from matplotlib import pyplot as plt
import seaborn as sns


In [3]:
get_digit = lambda x: int(''.join(list(filter(str.isdigit, x))))
def filename_parser(filename):
    info_str = filename.split('.')[0].split('_')[2:-1]
    chunk = get_digit(info_str[0])
    stride = get_digit(info_str[1])
    bound = get_digit(info_str[2])
    scale = 1 if info_str[3] == 'nonpool' else get_digit(info_str[3])
    return chunk, stride, bound, scale

In [9]:
deephic_file = "example_chr22_raw_downratio16.npz"
deephic_data = np.load("/share/home/mliu/shareb/mliu/evaluate_impute/tools/DeepHiC/server/example_chr22_raw_downratio16.npz", allow_pickle=True)

In [55]:
deephic_data.files
deephic_data['hic'].shape

(5136, 5136)

In [7]:
#加载pth模型
model = torch.load("save/deephic_raw_100.pth", map_location=torch.device('cpu'))
ckpt_file = "save/deephic_raw_100.pth"

In [13]:
#生成20*20的随机矩阵
matrix = np.random.rand(20, 20) * 2 - 1

In [28]:
matrix = matrix.reshape(1,20,20)

In [36]:
dataset = TensorDataset(torch.from_numpy(matrix))
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
dataloader[0].shape


TypeError: 'DataLoader' object is not subscriptable

In [5]:
def predict(inputs,ckpt_file,device):
    inputs  = inputs.reshape(1,1,inputs.shape[0],inputs.shape[1])
    dataset = TensorDataset(torch.from_numpy(inputs).float())
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
    scale = 1
    deepmodel = deephic.Generator(scale_factor=scale, in_channel=1, resblock_num=5).to(device)
    deepmodel.load_state_dict(torch.load(ckpt_file, map_location=torch.device('cpu')))

    deepmodel.eval()
    with torch.no_grad():
        for lr in dataloader:
            lr = lr[0].to(device)
            out = deepmodel(lr)
    out_npy = out.to('cpu').numpy()
    out_npy = np.squeeze(out_npy)
    print(out_npy.shape)
    return out_npy


In [2]:

target = "Cell2020"
imputed_matrix_dir = f"/shareb/mliu/evaluate_impute/data/imputed_data/deephic"

In [66]:

print(target)
downsample_matrix_dir = f"/shareb/mliu/evaluate_impute/data/simulation_hic/{target}/hic/matrix"
for group in range(1,5):
    print(group)
    downsample_matrix = np.load(f"{downsample_matrix_dir}/{target}_sample_matrix{group}.npy")
    downsample_matrix = downsample_matrix[0:100,:]
    imputed_matrices = np.array([predict(mat,ckpt_file,"cpu") for mat in downsample_matrix])

    for mat in imputed_matrices:
        np.fill_diagonal(mat, np.nan)

    print(imputed_matrices.shape)
    #保存
    save_path = f"{imputed_matrix_dir}/deephic_imputed_{target}_group{group}.npy"
    print(save_path)
    np.save(save_path, imputed_matrices)


Cell2020
1
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)
(243, 243)

## Real data

In [8]:
chr_list = ["chr1","chr2","chr3","chr4","chr5","chr6","chr7","chr8","chr9","chr10","chr11","chr12","chr13","chr14","chr15","chr16","chr17","chr18","chr19","chr20","chr21","chr22"]
sc_matrix_dir = f"/shareb/mliu/evaluate_impute/data/real_sc_hic/Ramani/cooler"
for chr in chr_list:
    print(chr)
    imputed_matrices_list = []
    for cell in range(0,620):
        print(f"cell{cell}")
        clr = cooler.Cooler(f"{sc_matrix_dir}/cell{cell}.cool")
        matrix_chr = clr.matrix(balance=False).fetch(chr)
        matrix_chr_imputed = predict(matrix_chr,ckpt_file,"cpu")
        imputed_matrices_list.append(matrix_chr_imputed)
    imputed_matrices_array = np.array(imputed_matrices_list)
    print(imputed_matrices_array.shape)
    save_path = f"{imputed_matrix_dir}/schicluster_imputed_Ramani_{chr}.npy"
    np.save(save_path, imputed_matrices_array)

chr1
cell0
(250, 250)
cell1
(250, 250)
cell2
(250, 250)
cell3
(250, 250)
cell4
(250, 250)
cell5
(250, 250)
cell6
(250, 250)
cell7
(250, 250)
cell8
(250, 250)
cell9
(250, 250)
cell10
(250, 250)
cell11
(250, 250)
cell12
(250, 250)
cell13
(250, 250)
cell14
(250, 250)
cell15
(250, 250)
cell16
(250, 250)
cell17
(250, 250)
cell18
(250, 250)
cell19
(250, 250)
cell20
(250, 250)
cell21
(250, 250)
cell22
(250, 250)
cell23
(250, 250)
cell24
(250, 250)
cell25
(250, 250)
cell26
(250, 250)
cell27
(250, 250)
cell28
(250, 250)
cell29
(250, 250)
cell30
(250, 250)
cell31
(250, 250)
cell32
(250, 250)
cell33
(250, 250)
cell34
(250, 250)
cell35
(250, 250)
cell36
(250, 250)
cell37
(250, 250)
cell38
(250, 250)
cell39
(250, 250)
cell40
(250, 250)
cell41
(250, 250)
cell42
(250, 250)
cell43
(250, 250)
cell44
(250, 250)
cell45
(250, 250)
cell46
(250, 250)
cell47
(250, 250)
cell48
(250, 250)
cell49
(250, 250)
cell50
(250, 250)
cell51
(250, 250)
cell52
(250, 250)
cell53
(250, 250)
cell54
(250, 250)
cell55
(250, 25